# Datos abiertos enero 2020

In [2]:
import os
import sys
import logging
import pandas as pd

from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'fredo7'
directory = 'BigData'

In [8]:
raw_data = get_input_data(
    bucket=bucket,
    initial_directory=directory,
    filename = 'datos-abiertos-enero_2020.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15304 entries, 0 to 15303
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NUMERO_INCIDENTE     15304 non-null  object 
 1   FECHA_DESPACHO_518   15304 non-null  object 
 2   CODIGO DE LOCALIDAD  15304 non-null  int64  
 3   LOCALIDAD            15304 non-null  object 
 4   EDAD                 15304 non-null  int64  
 5   UNIDAD               15304 non-null  object 
 6   GENERO               15304 non-null  object 
 7   RED                  15304 non-null  object 
 8   TIPO_INCIDENTE       15304 non-null  object 
 9   PRIORIDAD            15304 non-null  int64  
 10  Unnamed: 10          0 non-null      float64
 11  Unnamed: 11          0 non-null      float64
 12  Unnamed: 12          0 non-null      float64
 13  Unnamed: 13          0 non-null      float64
 14  Unnamed: 14          0 non-null      float64
 15  Unnamed: 15          0 non-null     

In [9]:
#Eliminamos todas las columnas Unnamed

list_cols_to_drop = ["Unnamed: {}".format(i) for i in range(10,37)]
list_cols_to_drop
raw_data = raw_data.drop(list_cols_to_drop, axis=1)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15304 entries, 0 to 15303
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     15304 non-null  object
 1   FECHA_DESPACHO_518   15304 non-null  object
 2   CODIGO DE LOCALIDAD  15304 non-null  int64 
 3   LOCALIDAD            15304 non-null  object
 4   EDAD                 15304 non-null  int64 
 5   UNIDAD               15304 non-null  object
 6   GENERO               15304 non-null  object
 7   RED                  15304 non-null  object
 8   TIPO_INCIDENTE       15304 non-null  object
 9   PRIORIDAD            15304 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 1.2+ MB


In [11]:
# Realizamos los cambios de nombres de acuerdo al estandar de cada archivo.

raw_data.rename(
    columns = {
        'FECHA_DESPACHO_518' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'TIPO DE INCIDIDENT': 'TIPO_INCIDENTE',
            },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2
1,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2
2,CRU-00000005-20,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1
3,CRU-00000024-20,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1
4,CRU-00000036-20,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1


In [12]:
# generamos una nueva columna con "Fecha_incidente" y la copiamos de
#fecha de desplazamiento movil.

raw_data = raw_data.assign(FECHA_INCIDENTE= raw_data['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'])
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,FECHA_INCIDENTE
0,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,2020-01-01 00:13:40
1,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,2020-01-01 00:13:40
2,CRU-00000005-20,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,2020-01-01 00:15:14
3,CRU-00000024-20,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,2020-01-01 00:20:50
4,CRU-00000036-20,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,2020-01-01 00:26:05


In [14]:
# ASIGNAMOS EL MES (ENERO - mes que estamos construyendo)
#Y LA CLASIFICACION FINAL (SIN DATOS).

raw_data = raw_data.assign( MES = 'ENERO')
raw_data.head()
raw_data = raw_data.assign(CLASIFICACION_FINAL = "SIN_DATO")
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,FECHA_INCIDENTE,MES,CLASIFICACION_FINAL
0,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,2020-01-01 00:13:40,ENERO,SIN_DATO
1,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,2020-01-01 00:13:40,ENERO,SIN_DATO
2,CRU-00000005-20,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,2020-01-01 00:15:14,ENERO,SIN_DATO
3,CRU-00000024-20,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,2020-01-01 00:20:50,ENERO,SIN_DATO
4,CRU-00000036-20,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,2020-01-01 00:26:05,ENERO,SIN_DATO


In [17]:
# Reordenamos las columnas en el orden establecido para todos
#los archivos

raw_data = raw_data.reindex(columns=['NUMERO_INCIDENTE',
                                     'FECHA_INCIDENTE',
'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
'CODIGO_LOCALIDAD',
'LOCALIDAD',
'EDAD',
'UNIDAD',
'GENERO',
'RED',
'TIPO_INCIDENTE',
'PRIORIDAD',
'MES',
'CLASIFICACION_FINAL',

])
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,CRU-00000002-20,2020-01-01 00:13:40,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,ENERO,SIN_DATO
1,CRU-00000002-20,2020-01-01 00:13:40,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2,ENERO,SIN_DATO
2,CRU-00000005-20,2020-01-01 00:15:14,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,ENERO,SIN_DATO
3,CRU-00000024-20,2020-01-01 00:20:50,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,ENERO,SIN_DATO
4,CRU-00000036-20,2020-01-01 00:26:05,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1,ENERO,SIN_DATO


In [18]:
# Columna NUMERO INCIDENTE
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['NUMERO_INCIDENTE'].value_counts(dropna=False)

CRU-00008619-20    14
CRU-00037891-20    14
CRU-00003298-20    13
CRU-00025187-20    13
CRU-00004092-20    12
                   ..
CRU-00013921-20     1
CRU-00013922-20     1
CRU-00013933-20     1
CRU-00013935-20     1
CRU-00037858-20     1
Name: NUMERO_INCIDENTE, Length: 13502, dtype: int64

In [19]:
raw_data['NUMERO_INCIDENTE'].apply(
    lambda x: x.strip()        
).value_counts()

CRU-00008619-20    14
CRU-00037891-20    14
CRU-00003298-20    13
CRU-00025187-20    13
CRU-00004092-20    12
                   ..
CRU-00013921-20     1
CRU-00013922-20     1
CRU-00013933-20     1
CRU-00013935-20     1
CRU-00037858-20     1
Name: NUMERO_INCIDENTE, Length: 13502, dtype: int64

In [20]:
# Columna FECHA INCIDENTE
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['FECHA_INCIDENTE'].value_counts(dropna=False)

2020-01-04 08:06:26    10
2020-01-29 12:51:15     8
2020-01-26 03:37:17     7
2020-01-25 18:08:28     7
2020-01-22 09:37:57     6
                       ..
2020-01-12 22:41:58     1
2020-01-12 22:44:20     1
2020-01-12 22:49:49     1
2020-01-12 22:54:18     1
2020-02-01 01:19:50     1
Name: FECHA_INCIDENTE, Length: 14958, dtype: int64

In [21]:
raw_data['FECHA_INCIDENTE'].apply(
    lambda x: x.strip()        
).value_counts()

2020-01-04 08:06:26    10
2020-01-29 12:51:15     8
2020-01-26 03:37:17     7
2020-01-25 18:08:28     7
2020-01-22 09:37:57     6
                       ..
2020-01-12 22:41:58     1
2020-01-12 22:44:20     1
2020-01-12 22:49:49     1
2020-01-12 22:54:18     1
2020-02-01 01:19:50     1
Name: FECHA_INCIDENTE, Length: 14958, dtype: int64

In [22]:
# Columna FECHA INCIDENTE
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].value_counts(dropna=False)

2020-01-04 08:06:26    10
2020-01-29 12:51:15     8
2020-01-26 03:37:17     7
2020-01-25 18:08:28     7
2020-01-22 09:37:57     6
                       ..
2020-01-12 22:41:58     1
2020-01-12 22:44:20     1
2020-01-12 22:49:49     1
2020-01-12 22:54:18     1
2020-02-01 01:19:50     1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 14958, dtype: int64

In [23]:
raw_data['FECHA_INICIO_DESPLAZAMIENTO_MOVIL'].apply(
    lambda x: x.strip()        
).value_counts()

2020-01-04 08:06:26    10
2020-01-29 12:51:15     8
2020-01-26 03:37:17     7
2020-01-25 18:08:28     7
2020-01-22 09:37:57     6
                       ..
2020-01-12 22:41:58     1
2020-01-12 22:44:20     1
2020-01-12 22:49:49     1
2020-01-12 22:54:18     1
2020-02-01 01:19:50     1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 14958, dtype: int64

In [24]:
# Columna CODIGO LOCALIDAD
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['CODIGO_LOCALIDAD'].value_counts(dropna=False)


8     2116
10    1473
11    1465
7     1072
19    1022
16     942
9      812
1      786
4      746
18     737
5      678
6      550
3      541
13     527
12     506
2      498
14     446
15     320
17      64
20       3
Name: CODIGO_LOCALIDAD, dtype: int64

In [25]:
# Columna LOCALIDAD
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['LOCALIDAD'].value_counts(dropna=False)


 Kennedy              2116
 Engativá             1473
 Suba                 1465
 Bosa                 1072
 Ciudad Bolívar       1022
 Puente Aranda         942
 Fontibón              812
Usaquén                786
 San Cristóbal         746
Rafael Uribe Uribe     737
Usme                   678
Tunjuelito             550
Santa Fe               541
 Teusaquillo           527
 Barrios Unidos        506
Chapinero              498
 Los Mártires          446
 Antonio Nariño        320
 La Candelaria          64
Sumapaz                  3
Name: LOCALIDAD, dtype: int64

In [26]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        
).value_counts()

Kennedy               2116
Engativá              1473
Suba                  1465
Bosa                  1072
Ciudad Bolívar        1022
Puente Aranda          942
Fontibón               812
Usaquén                786
San Cristóbal          746
Rafael Uribe Uribe     737
Usme                   678
Tunjuelito             550
Santa Fe               541
Teusaquillo            527
Barrios Unidos         506
Chapinero              498
Los Mártires           446
Antonio Nariño         320
La Candelaria           64
Sumapaz                  3
Name: LOCALIDAD, dtype: int64

In [27]:
# Columna EDAD
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['EDAD'].value_counts(dropna=False)


0     7004
23     245
25     233
24     225
22     220
      ... 
1       14
97      13
7       13
4       11
98      10
Name: EDAD, Length: 100, dtype: int64

In [29]:
# Columna UNIDAD
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['UNIDAD'].value_counts(dropna=False)


Años        8259
SIN_DATO    7003
Meses         34
Horas          5
Dias           3
Name: UNIDAD, dtype: int64

In [30]:
# Columna GENERO
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['GENERO'].value_counts(dropna=False)


SIN_DATO     7003
MASCULINO    4521
FEMENINO     3780
Name: GENERO, dtype: int64

In [31]:
# Columna RED
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['RED'].value_counts(dropna=False)


Norte      6155
Sur        5291
Privada    3858
Name: RED, dtype: int64

In [32]:
# Columna TIPO DE INCIDENTE
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['TIPO_INCIDENTE'].value_counts(dropna=False)


Heridos                                   6849
Inconsciente/Paro Cardiorrespiratorio     1520
Dificultad Respiratoria                   1359
Enfermo                                   1045
Convulsiones                               994
Trastorno mental                           661
Dolor torácico                             634
Intento de suicidio                        503
Accidente cerebro vascular                 374
Patología Gineco - obstétrica              331
Síntomas gastrointestinales                331
Caída                                      320
Intoxicaciones                             138
Ideas de suicidio                           99
SIN_DATO                                    80
Violencia Sexual                            34
Quemaduras                                  10
Sangrado Vaginal                             8
Maltrato                                     7
Electrocución / Rescate                      7
Name: TIPO_INCIDENTE, dtype: int64

In [33]:
raw_data['TIPO_INCIDENTE'].apply(
    lambda x: x.strip()        
).value_counts()

Heridos                                  6849
Inconsciente/Paro Cardiorrespiratorio    1520
Dificultad Respiratoria                  1359
Enfermo                                  1045
Convulsiones                              994
Trastorno mental                          661
Dolor torácico                            634
Intento de suicidio                       503
Accidente cerebro vascular                374
Patología Gineco - obstétrica             331
Síntomas gastrointestinales               331
Caída                                     320
Intoxicaciones                            138
Ideas de suicidio                          99
SIN_DATO                                   80
Violencia Sexual                           34
Quemaduras                                 10
Sangrado Vaginal                            8
Maltrato                                    7
Electrocución / Rescate                     7
Name: TIPO_INCIDENTE, dtype: int64

In [34]:
# Columna PRIORIDAD
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['PRIORIDAD'].value_counts(dropna=False)



2    10084
1     3017
3     2048
4      155
Name: PRIORIDAD, dtype: int64

In [35]:
# Columna MES
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['MES'].value_counts(dropna=False)


ENERO    15304
Name: MES, dtype: int64

In [36]:
# Columna CLASIFICACION FINAL
# Se validan hay datos nullos 
# Si hay datos nullos se reemplazan por SIN_DATO
# Retiramos todos los espacios en blanco que existan
raw_data['CLASIFICACION_FINAL'].value_counts(dropna=False)


SIN_DATO    15304
Name: CLASIFICACION_FINAL, dtype: int64

In [37]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_enero_2020.html')

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 15304
Number of rows after cleaning: 15240


In [39]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_enero_2020.csv', encoding='latin1', sep=';', index=False)